In [2]:
import numpy as np 
import csv 
import glob 
import os
import re
from collections import defaultdict
import torch
import h5py

In [3]:
window_size = 1000
threshold = 60
slide_size = 200

In [4]:
def dataimport(path1, path2):

	xx = np.empty([0,window_size,90],float)
	yy = np.empty([0,8],float)

	###Input data###
	#data import from csv
	input_csv_files = sorted(glob.glob(path1))
	for f in input_csv_files:
		print("input_file_name=",f)
		data = [[ float(elm) for elm in v] for v in csv.reader(open(f, "r"))]
		tmp1 = np.array(data)
		x2 =np.empty([0,window_size,90],float)

		#data import by slide window
		k = 0
		while k <= (len(tmp1) + 1 - 2 * window_size):
			x = np.dstack(np.array(tmp1[k:k+window_size, 1:91]).T)
			x2 = np.concatenate((x2, x),axis=0)
			k += slide_size

		xx = np.concatenate((xx,x2),axis=0)
	xx = xx.reshape(len(xx),-1)

	###Annotation data###
	#data import from csv
	annotation_csv_files = sorted(glob.glob(path2))
	for ff in annotation_csv_files:
		print("annotation_file_name=",ff)
		ano_data = [[ str(elm) for elm in v] for v in csv.reader(open(ff,"r"))]
		tmp2 = np.array(ano_data)

		#data import by slide window
		y = np.zeros(((len(tmp2) + 1 - 2 * window_size)//slide_size+1,8))
		k = 0
		while k <= (len(tmp2) + 1 - 2 * window_size):
			y_pre = np.stack(np.array(tmp2[k:k+window_size]))
			bed = 0
			fall = 0
			walk = 0
			pickup = 0
			run = 0
			sitdown = 0
			standup = 0
			noactivity = 0
			for j in range(window_size):
				if y_pre[j] == "bed":
					bed += 1
				elif y_pre[j] == "fall":
					fall += 1
				elif y_pre[j] == "walk":
					walk += 1
				elif y_pre[j] == "pickup":
					pickup += 1
				elif y_pre[j] == "run":
					run += 1
				elif y_pre[j] == "sitdown":
					sitdown += 1
				elif y_pre[j] == "standup":
					standup += 1
				else:
					noactivity += 1

			if bed > window_size * threshold / 100:
				y[k/slide_size,:] = np.array([0,1,0,0,0,0,0,0])
			elif fall > window_size * threshold / 100:
				y[k/slide_size,:] = np.array([0,0,1,0,0,0,0,0])
			elif walk > window_size * threshold / 100:
				y[k/slide_size,:] = np.array([0,0,0,1,0,0,0,0])
			elif pickup > window_size * threshold / 100:
				y[k/slide_size,:] = np.array([0,0,0,0,1,0,0,0])
			elif run > window_size * threshold / 100:
				y[k/slide_size,:] = np.array([0,0,0,0,0,1,0,0])
			elif sitdown > window_size * threshold / 100:
				y[k/slide_size,:] = np.array([0,0,0,0,0,0,1,0])
			elif standup > window_size * threshold / 100:
				y[k/slide_size,:] = np.array([0,0,0,0,0,0,0,1])
			else:
				y[k/slide_size,:] = np.array([2,0,0,0,0,0,0,0])
			k += slide_size

		yy = np.concatenate((yy, y),axis=0)
	print(xx.shape,yy.shape)
	return (xx, yy)


In [5]:
input_path = "../data/UT_HAR_OG/input"
annotation_path = "../data/UT_HAR_OG/annotation"

In [6]:
input_files = os.listdir(input_path)
annotation_files = os.listdir(annotation_path)

def extract_keys(input_files, annotation_files):
    # Define patterns for the two naming conventions
    pattern_activity = r"input_([a-z]+_\d+_\d+_\d+)"  # Matches input_ACTIVITY_XXXX_XXXX_XX
    pattern_name = r"input_\d+_([a-z]+_[a-z]+_\d+)"    # Matches input_XXXX_NAME_ACTIVITY_XX

    # Dictionary to hold associations
    file_associations = defaultdict(lambda: {"input": None, "annotation": None})

    # Process input files
    for file in input_files:
        match_activity = re.match(pattern_activity, file)
        match_name = re.match(pattern_name, file)
        if match_activity:
            key = match_activity.group(1)
            file_associations[key]["input"] = file
        elif match_name:
            key = match_name.group(1)
            file_associations[key]["input"] = file

    # Process annotation files
    for file in annotation_files:
        # Extract keys based on the annotation naming conventions
        match_activity = re.match(r"annotation_([a-z]+_\d+_\d+_\d+)", file)
        match_name = re.match(r"annotation_([a-z]+_[a-z]+_\d+)", file)
        if match_activity:
            key = match_activity.group(1)
            file_associations[key]["annotation"] = file
        elif match_name:
            key = match_name.group(1)
            file_associations[key]["annotation"] = file

    # Convert defaultdict to regular dictionary for final output
    return dict(file_associations)

# Print associations
file_associations = extract_keys(input_files, annotation_files)
for key, value in file_associations.items():
    print(f"{key}: {value}")

pickup_170309_1240_07: {'input': 'input_pickup_170309_1240_07.csv', 'annotation': 'annotation_pickup_170309_1240_07.csv'}
sankalp_bed_1: {'input': 'input_161219_sankalp_bed_1.csv', 'annotation': 'annotation_sankalp_bed_1.csv'}
sankalp_bed_10: {'input': 'input_161219_sankalp_bed_10.csv', 'annotation': 'annotation_sankalp_bed_10.csv'}
sankalp_bed_2: {'input': 'input_161219_sankalp_bed_2.csv', 'annotation': 'annotation_sankalp_bed_2.csv'}
sankalp_bed_3: {'input': 'input_161219_sankalp_bed_3.csv', 'annotation': 'annotation_sankalp_bed_3.csv'}
sankalp_bed_4: {'input': 'input_161219_sankalp_bed_4.csv', 'annotation': 'annotation_sankalp_bed_4.csv'}
sankalp_bed_5: {'input': 'input_161219_sankalp_bed_5.csv', 'annotation': 'annotation_sankalp_bed_5.csv'}
sankalp_bed_6: {'input': 'input_161219_sankalp_bed_6.csv', 'annotation': 'annotation_sankalp_bed_6.csv'}
sankalp_bed_7: {'input': 'input_161219_sankalp_bed_7.csv', 'annotation': 'annotation_sankalp_bed_7.csv'}
sankalp_bed_8: {'input': 'input_161

In [7]:
input_files = os.listdir(input_path)
annotation_files = os.listdir(annotation_path) 
file_associations = extract_keys(input_files, annotation_files) 

len(file_associations)

557

In [8]:
def create_dataset(input_path, annotation_path):
    input_files = os.listdir(input_path)
    annotation_files = os.listdir(annotation_path) 
    file_associations = extract_keys(input_files, annotation_files) 

    #create np array that will store the attributes in the format [capture, rows, columns]
    #only store the rows that have an associated annotation different from 'NoActivity'
    #ignore 1st column from input file since it contains timestamps
    #create vector that will store the labels
    data = []
    labels = []
    idx = 0
    for capture, files in file_associations.items():
        input_file = files["input"]
        annotation_file = files["annotation"]

        if not input_file or not annotation_file:
            # Skip if either input or annotation file is missing
            continue

        # Load input and annotation files
        input_data = np.loadtxt(os.path.join(input_path, input_file), delimiter=',')
        annotation_data = np.loadtxt(os.path.join(annotation_path, annotation_file), delimiter=',', dtype=str)

        # Check if input and annotation file sizes match
        if input_data.shape[0] != annotation_data.shape[0]:
            print(f"Skipping {capture}: Mismatch in row counts between input and annotation files.")
            continue

        # Ignore the first column (timestamps) in input file
        input_data = input_data[:, 1:]

        # Iterate over rows and filter based on annotation
        capture_data = [] 
        for i in range(annotation_data.shape[0]):
            if annotation_data[i] != 'NoActivity':  # Assume the first column in annotation contains labels
                capture_data.append(torch.tensor(input_data[i,:],dtype=torch.float32))
                label = annotation_data[i]
        data.append(torch.stack(capture_data)) 
        labels.append(label)

    # Convert data and labels to numpy arrays
    labels = np.array(labels)

    return data, labels


In [9]:
data, labels = create_dataset(input_path, annotation_path)

KeyboardInterrupt: 

In [ ]:
len(data)

557

In [ ]:
data[3].shape

torch.Size([3531, 180])

In [ ]:
labels.shape

(557,)

In [ ]:
def pad_to_3d(data):
    max_rows = max(tensor.size(0) for tensor in data)

    padded_tensors = []

    for tensor in data:

        padding_rows = max_rows - tensor.size(0)
        padded_tensor = torch.nn.functional.pad(tensor,(0,0,0,padding_rows))
        padded_tensors.append(padded_tensor)

    return torch.stack(padded_tensors)

In [ ]:
padded_data = pad_to_3d(data)

In [ ]:
padded_data.shape

torch.Size([557, 11301, 180])

In [ ]:
file_associations.items()

dict_items([('pickup_170309_1240_07', {'input': 'input_pickup_170309_1240_07.csv', 'annotation': 'annotation_pickup_170309_1240_07.csv'}), ('sankalp_bed_1', {'input': 'input_161219_sankalp_bed_1.csv', 'annotation': 'annotation_sankalp_bed_1.csv'}), ('sankalp_bed_10', {'input': 'input_161219_sankalp_bed_10.csv', 'annotation': 'annotation_sankalp_bed_10.csv'}), ('sankalp_bed_2', {'input': 'input_161219_sankalp_bed_2.csv', 'annotation': 'annotation_sankalp_bed_2.csv'}), ('sankalp_bed_3', {'input': 'input_161219_sankalp_bed_3.csv', 'annotation': 'annotation_sankalp_bed_3.csv'}), ('sankalp_bed_4', {'input': 'input_161219_sankalp_bed_4.csv', 'annotation': 'annotation_sankalp_bed_4.csv'}), ('sankalp_bed_5', {'input': 'input_161219_sankalp_bed_5.csv', 'annotation': 'annotation_sankalp_bed_5.csv'}), ('sankalp_bed_6', {'input': 'input_161219_sankalp_bed_6.csv', 'annotation': 'annotation_sankalp_bed_6.csv'}), ('sankalp_bed_7', {'input': 'input_161219_sankalp_bed_7.csv', 'annotation': 'annotation_

In [ ]:
labels

array(['pickup', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed', 'bed',
       'bed', 'bed', 'fall', 'fall', 'fall', 'pickup', 'pickup', 'pickup',
       'pickup', 'pickup', 'pickup', 'pickup', 'pickup', 'pickup',
       'pickup', 'pickup', 'pickup', 'pickup', 'pickup', 'fall', 'fall',
       'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'fall',
       'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'sitdown',
       'sitdown', 'sitdown', 'sitdown', 'sitdown', 'sitdown', 'sitdown',
       'sitdown', 'sitdown', 'sitdown', 'sitdown', 'sitdown', 'sitdown',
       'sitdown', 'sitdown', 'sitdown', 'sitdown', 'fall', 'pickup',
       'standup', 'bed', 'pickup', 'sitdown', 'walk', 'bed', 'bed',
       'fall', 'fall', 'fall', 'pickup', 'pickup', 'pickup', 'pickup',
       'pickup', 'pickup', 'run', 'run', 'run', 'run', 'run', 'run',
       'run', 'run', 'run', 'run', 'sitdown', 'sitdown', 'sitdown',
       'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk',
       'walk'

In [ ]:
padded_data[0]

tensor([[11.6940, 12.9730, 14.2000,  ..., 12.3690, 12.2050, 12.5090],
        [11.6940, 12.9730, 14.2000,  ..., 12.3690, 12.2050, 12.5090],
        [ 9.9784, 12.8560, 14.5720,  ..., 13.0370, 12.8420, 13.1860],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

## Motivating the Usage of CSI Ratio for Preprocessing CSI Data

### Motivation
The **Channel State Information (CSI)** contains detailed amplitude and phase data about the wireless channel. While useful, CSI data often suffers from significant challenges that limit its direct application in sensing tasks:

1. **Noisy Phase Data**:
   - CSI phase is prone to instability caused by hardware imperfections such as carrier frequency offset (CFO), sampling frequency offset (SFO), and time-varying random phase offsets.
   - These phase errors reduce the reliability of using raw CSI data.

2. **Environmental Noise**:
   - CSI amplitude is affected by environmental changes, including large-scale fading and multipath effects, making it harder to detect subtle signal variations.

3. **Device Dependency**:
   - Absolute CSI values are dependent on hardware characteristics like power amplifier gains, which may vary across devices, creating inconsistencies in results.

4. **Limited Sensing Range**:
   - Noise and large-scale variations restrict the range at which the system can detect meaningful information.

The **CSI ratio** technique is a preprocessing method that addresses these issues by normalizing the data through division operations between antennas. It effectively mitigates noise, cancels out phase offsets, and emphasizes subtle channel variations while suppressing large-scale disturbances. This makes it particularly useful for tasks requiring precise detection, such as respiration monitoring or activity recognition.

---

### How CSI Ratio is Performed

To compute the **CSI ratio**, follow these steps:

1. **Collect CSI Data**:
   - Acquire raw CSI data from multiple antennas. For $ N $ antennas, the CSI values are represented as complex numbers $ H_i $, where $ i $ ranges from $ 1 $ to $ N $.

2. **Pair Antennas**:
   - Compute ratios between pairs of antennas. For $ N $ antennas, there are $ \binom{N}{2} $ unique pairs. Each pair $ (i, j) $ generates a ratio:
   $$ 
     H_{i,j} = \frac{H_i}{H_j}
   $$

3. **Decompose the Ratio**:
   - The ratio $ H_{i,j} $ is a complex number and can be expressed as:
   $$
     H_{i,j} = A_{i,j} e^{j \Delta \phi_{i,j}}
   $$
     Where:
     - $ A_{i,j} = \frac{|H_i|}{|H_j|} $ is the amplitude ratio.
     - $ \Delta \phi_{i,j} = \phi_i - \phi_j $ is the phase difference.

4. **Cancel Noise**:
   - The division cancels out time-varying random offsets and hardware-induced phase errors because these offsets are the same across antennas in a single device. This provides a **cleaner and more stable signal**.

5. **Enhance Sensitivity**:
   - The resulting amplitude ratios and phase differences are normalized, reducing sensitivity to large-scale fading and amplifying subtle variations caused by local movements.

6. **Combine Amplitude and Phase** (Optional):
   - The orthogonal properties of amplitude and phase can be combined to enhance sensing capabilities further. For example, projecting the complex CSI ratio onto multiple axes in the complex plane extracts richer information.

---

### Benefits of CSI Ratio Preprocessing
- **Noise Suppression**: Division cancels out shared noise components and random phase offsets.
- **Enhanced Sensitivity**: Highlights subtle changes in the environment, such as human respiration.
- **Increased Range**: Allows for sensing over greater distances by stabilizing phase and reducing noise.
- **Device Independence**: Normalized ratios are less affected by hardware-dependent variations.
- **Robustness**: More resilient to multipath effects and large-scale environmental changes.

By preprocessing raw CSI data with the CSI ratio technique, researchers can significantly improve the reliability and accuracy of wireless sensing systems for diverse applications.


In [ ]:
padded_data[0,:,int(padded_data[0,0,:].shape[0]/2)-1:-1].shape

torch.Size([11301, 90])

In [ ]:
def cvt_to_csi_ratio(data):
    proc_data = torch.zeros_like(data)
    for idx, tensor in enumerate(data):
        num_antennas = 3
        num_subcarriers = 30
        A_n = np.roll(tensor[:,:num_antennas*num_subcarriers], shift=num_subcarriers, axis=1)
        A_n[:,:num_subcarriers] = tensor[:,:num_subcarriers]
        A_d = np.roll(tensor[:,:num_antennas*num_subcarriers], shift=-num_subcarriers, axis=1)
        A_d[:,2*num_subcarriers:3*num_subcarriers] = tensor[:,2*num_subcarriers:3*num_subcarriers]

        theta_n = np.roll(tensor[:,num_antennas*num_subcarriers:], shift=num_subcarriers, axis=1)
        theta_n[:,:num_subcarriers] = tensor[:,num_antennas*num_subcarriers:(num_antennas+1)*num_subcarriers]
        theta_d = np.roll(tensor[:,num_antennas*num_subcarriers:], shift=-num_subcarriers, axis=1)
        theta_d[:,2*num_subcarriers:3*num_subcarriers] = tensor[:,(num_antennas+2)*num_subcarriers:(num_antennas+3)*num_subcarriers]

        proc_data[idx, :, :num_antennas*num_subcarriers] = torch.tensor(np.divide(A_n, A_d, where=A_d!=0))
        proc_data[idx, :, num_antennas*num_subcarriers:] = torch.tensor(theta_n - theta_d)

    return proc_data

In [ ]:
test = cvt_to_csi_ratio(padded_data[0:1])

/tmp/ipykernel_67756/2377619306.py:7: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  A_n[:,:num_subcarriers] = tensor[:,:num_subcarriers]
/tmp/ipykernel_67756/2377619306.py:9: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  A_d[:,2*num_subcarriers:3*num_subcarriers] = tensor[:,2*num_subcarriers:3*num_subcarriers]
/tmp/ipykernel_67756/2377619306.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  theta_n[:,:num_subcarriers] = tensor[:,num_antennas*num_subcarriers:(num_antennas+1)*num_subcarriers]
/tmp/ipykernel_67756/2377619306.py:14: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing 

In [ ]:
csi_ratio = cvt_to_csi_ratio(padded_data)

/tmp/ipykernel_67756/2377619306.py:7: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  A_n[:,:num_subcarriers] = tensor[:,:num_subcarriers]
/tmp/ipykernel_67756/2377619306.py:9: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  A_d[:,2*num_subcarriers:3*num_subcarriers] = tensor[:,2*num_subcarriers:3*num_subcarriers]
/tmp/ipykernel_67756/2377619306.py:12: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  theta_n[:,:num_subcarriers] = tensor[:,num_antennas*num_subcarriers:(num_antennas+1)*num_subcarriers]
/tmp/ipykernel_67756/2377619306.py:14: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing 

In [ ]:
unique_labels = sorted(set(labels))
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
encoded_labels = [label_mapping[label] for label in labels]

print("Original Labels:", labels)
print("Encoded Labels:", encoded_labels)
print("Label Mapping:", label_mapping)

Original Labels: ['pickup' 'bed' 'bed' 'bed' 'bed' 'bed' 'bed' 'bed' 'bed' 'bed' 'bed'
 'fall' 'fall' 'fall' 'pickup' 'pickup' 'pickup' 'pickup' 'pickup'
 'pickup' 'pickup' 'pickup' 'pickup' 'pickup' 'pickup' 'pickup' 'pickup'
 'pickup' 'fall' 'fall' 'fall' 'fall' 'fall' 'fall' 'fall' 'fall' 'fall'
 'fall' 'fall' 'fall' 'fall' 'fall' 'fall' 'fall' 'sitdown' 'sitdown'
 'sitdown' 'sitdown' 'sitdown' 'sitdown' 'sitdown' 'sitdown' 'sitdown'
 'sitdown' 'sitdown' 'sitdown' 'sitdown' 'sitdown' 'sitdown' 'sitdown'
 'sitdown' 'fall' 'pickup' 'standup' 'bed' 'pickup' 'sitdown' 'walk' 'bed'
 'bed' 'fall' 'fall' 'fall' 'pickup' 'pickup' 'pickup' 'pickup' 'pickup'
 'pickup' 'run' 'run' 'run' 'run' 'run' 'run' 'run' 'run' 'run' 'run'
 'sitdown' 'sitdown' 'sitdown' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk'
 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'walk' 'pickup'
 'pickup' 'pickup' 'pickup' 'pickup' 'pickup' 'pickup' 'pickup' 'pickup'
 'pickup' 'pickup' 'pickup' 'pickup' 'pickup' 'pick

In [ ]:
with h5py.File("../data/UT_HAR_OG/X.h5","w") as f:
    f.create_dataset("X", data=padded_data)
with h5py.File("../data/UT_HAR_OG/y.h5","w") as f:
    f.create_dataset("y", data=encoded_labels)

In [ ]:
with h5py.File("../data/UT_HAR_CSI_RATIO/X.h5","w") as f:
    f.create_dataset("X", data=csi_ratio)
with h5py.File("../data/UT_HAR_CSI_RATIO/y.h5","w") as f:
    f.create_dataset("y", data=encoded_labels)

: 